In [5]:
from __future__ import print_function
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation,Flatten,BatchNormalization
from keras.layers import Conv2D,MaxPooling2D
import os
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

num_classes = 5
img_rows,img_cols = 48,48
batch_size = 32

train_data_dir = 'C:/Users/lenovo/Desktop/my code/my work/emoition/fer2013/train'
validation_data_dir = 'C:/Users/lenovo/Desktop/my code/my work/emoition/fer2013/validation'

train_datagen = ImageDataGenerator(rescale=1./255,
                                    rotation_range=30,
                                    shear_range=0.3,
                                    zoom_range=0.3,
                                    width_shift_range=0.4,
                                    height_shift_range=0.4,
                                    horizontal_flip=True,
                                    fill_mode='nearest')

validation_datagen = ImageDataGenerator(rescale=1./255)  

train_generator = train_datagen.flow_from_directory(train_data_dir,
                                                    color_mode='grayscale',
                                                    target_size=(img_rows,img_cols),
                                                    batch_size=batch_size,
                                                    class_mode='categorical',
                                                    shuffle=True)

validation_generator = validation_datagen.flow_from_directory(validation_data_dir,
                                                            color_mode='grayscale',
                                                            target_size=(img_rows,img_cols),
                                                            batch_size=batch_size,
                                                            class_mode='categorical',
                                                            shuffle=True) 

model = Sequential()

#block 1

model.add(Conv2D(32,(3,3),padding='same',kernel_initializer='he_normal',input_shape=(img_rows,img_cols,1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(32,(3,3),padding='same',kernel_initializer='he_normal',input_shape=(img_rows,img_cols,1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

#block 2

model.add(Conv2D(32,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(32,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

#block 3

model.add(Conv2D(128,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(128,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

#block 4 

model.add(Conv2D(256,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(256,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

#block 5
model.add(Flatten())
model.add(Dense(64,kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

#block 6
model.add(Dense(64,kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

#block 7
model.add(Dense(num_classes,kernel_initializer='he_normal'))
model.add(Activation('softmax'))

print(model.summary())

from keras.optimizers import RMSprop, SGD, Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

checkpoint = ModelCheckpoint('emotion_vgg.h5',
                              monitor='val_loss',
                              mode='min',
                              save_best_only=True,
                              verbose=1)

earlystop = EarlyStopping(monitor='val_loss',
                      min_delta=0,
                      patience=9,
                      verbose=1,
                      restore_best_weights=True)

reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.2,
                              patience=3,
                              verbose=1,
                              min_delta=0.0001)

callbacks = [earlystop,checkpoint,reduce_lr]

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.003),
              metrics=['accuracy'])

nb_train_samples = 24256
nb_validation_samples = 3006
epochs = 25

history = model.fit_generator(train_generator,
                              steps_per_epoch=nb_train_samples//batch_size,
                              epochs=epochs,
                              callbacks=callbacks,
                              validation_data=validation_generator,
                              validation_steps=nb_validation_samples//batch_size)

Found 24256 images belonging to 5 classes.
Found 3006 images belonging to 5 classes.
Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_33 (Conv2D)           (None, 48, 48, 32)        320       
_________________________________________________________________
activation_45 (Activation)   (None, 48, 48, 32)        0         
_________________________________________________________________
batch_normalization_41 (Batc (None, 48, 48, 32)        128       
_________________________________________________________________
conv2d_34 (Conv2D)           (None, 48, 48, 32)        9248      
_________________________________________________________________
activation_46 (Activation)   (None, 48, 48, 32)        0         
_________________________________________________________________
batch_normalization_42 (Batc (None, 48, 48, 32)        128       
___________________________________

758/758 [==============================] - 71s 94ms/step - loss: 1.2398 - acc: 0.4873 - val_loss: 1.2919 - val_acc: 0.4963oss: 1.2397 - acc: 0 - ETA: 0s - loss: 1.2400 - a

Epoch 00008: val_loss improved from 1.31758 to 1.29185, saving model to emotion_vgg.h5
Epoch 9/25
758/758 [==============================] - 71s 94ms/step - loss: 1.2039 - acc: 0.5107 - val_loss: 1.5044 - val_acc: 0.5158TA:

Epoch 00009: val_loss did not improve from 1.29185
Epoch 10/25
758/758 [==============================] - 71s 94ms/step - loss: 1.1844 - acc: 0.5188 - val_loss: 1.3687 - val_acc: 0.54241s - loss: 

Epoch 00010: val_loss did not improve from 1.29185
Epoch 11/25
758/758 [==============================] - 73s 96ms/step - loss: 1.1639 - acc: 0.5282 - val_loss: 1.3038 - val_acc: 0.5256

Epoch 00011: val_loss did not improve from 1.29185

Epoch 00011: ReduceLROnPlateau reducing learning rate to 0.0006000000052154065.
Epoch 12/25
758/758 [==============================] - 73s 96ms/step - loss: 1.1179 -